# Drive Train Parameters

As we design the alt-az mount, there are a bunch of options for designs, and the decision comes down to what kind of drive train we want to use.  That decision is in turn driven by the tracking precision needed in our system.  That, in turn, is decided by the optical parameters for exposure.

First up, our field of view: this is a function of the F.O.V. (aka Angle Of View) equation: $\frac{w}{f}\cdot\frac{180}{\pi}$, with $w$ the width of the sensor, $f$ the focal length of the scope, and an angle small enough that the small angle approximation holds (otherwise there's a bit of trig in there).  In our case, $f = 1000\textrm{mm}$, so this turns into simply $w_m \textrm{rad}$.  For the Canon EOS APS-C sensor, $w = (22.3,14.9)\textrm{mm}$, which leads to a FOV of (1.28°×0.85°).

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
f_mm = 1000
w_mm = np.array([22.3, 14.9])

fov_deg = w_mm/f_mm * 180.0/np.pi
fov_deg

array([ 1.27769588,  0.85370711])

The next thing to consider is the pixels at work: our sensor is 4,752×3,168, which leads to a pixel pitch of 4.69µm in both directions.  We can either divide the FOV from the previous step by the pixels, or just use the pixel pitch.  Either way, we get an angular resolution of 0.97 arcseconds per pixel on our sensor.

In [9]:
w_px = np.array([4752,3168])
angres1_px = fov_deg/w_px * 3600

angres_px = 4.69e-6*180/np.pi * 3600

angres1_px, angres_px

(array([ 0.96795143,  0.97012172]), 0.9673819412988819)

However, we'd be thoroughly blessed to get seeing that's better than 3", so 0.97"/pixel means that our average feature is roughly a 3×3 pixel blotch on the sensor, which we'll factor in later.  For now, let's find out how far the objects are going to be moving, in pixels, during each exposure.

We need to pull the average azimuthal speeds from `passes_to_parameters.ipynb` in this directory.  There, we get azimuthal rates with a median of 0.03573°/s and a mean of 0.152081°/s.  Applying Tukey inner fences, we get a mean of 0.05°/s, so the 0.15°/s is a pretty aggressive pass.  However, looking at ISS data, it seems that our mean is closer to 0.2°/s, so let's shoot for that.

There is a wide range of exposure options here, so we'll look at that as well.

In [15]:
speed_degps = 0.2

# This gets more complicated with resolutions, though, so we'll factor that in

# Webcam looks to be a 1/3" display: 4.8x3.6mm
wcw_px = 4.8
wch_px = 3.6

f_mm = 1000.0

cases = [
    ("Canon 500D 1080p", 1920,1080, 22.3,14.9, 20.0),
    ("Canon 500D 720p",  1280, 720, 22.3,14.9, 30.0),
    ("Canon 500D 1/3",   4752,3168, 22.3,14.9,  3.0),
    ("Webcam VGA",        640, 480, wcw_px,wch_px, 30.0),
    ("Webcam SVGA",       800, 600, wcw_px,wch_px, 20.0),
    ("Webcam UVGA",       1280, 960, wcw_px,wch_px,  9.0),
    ("Webcam UXGA",      1600,1200, wcw_px,wch_px,  5.0),
]


for camera, w_px,h_px, w_mm, h_mm, fps in cases:
    fov_degpp = (w_mm/w_px)/f_mm * 180/np.pi
    fov_deg = w_mm/f_mm * 180/np.pi
    print camera, fps, speed_degps/fov_deg, speed_degps/fov_degpp/fps

Canon 500D 1080p 20.0 0.156531771479 15.027050062
Canon 500D 720p 30.0 0.156531771479 6.67868891645
Canon 500D 1/3 3.0 0.156531771479 247.946326023
Webcam VGA 30.0 0.727220521664 15.5140377955
Webcam SVGA 20.0 0.727220521664 29.0888208666
Webcam UVGA 9.0 0.727220521664 103.426918637
Webcam UXGA 5.0 0.727220521664 232.710566933


I'm starting to think that this is all inside out, and I should be starting with the precision of the TLEs and sorting out the range of field I need to be able to see in order to best assess their validity… the above is all abou the direct imaging case, but that's possibly beyond my current equipment anyway.

In [20]:
np.array([4.8,3.6,0.003])/1000 * 180.0/np.pi * 3600

array([  9.90071070e+02,   7.42553302e+02,   6.18794419e-01])